### Importación de Librerías

In [58]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight

### Carga de datos

In [59]:
train = pd.read_csv('fetal_health.csv')

### Inspección inicial de datos  
Variable dependiente = fetal_health  
Posibles valores:  
* 1 : Normal
* 2 : Suspect
* 3 : Pathological

In [60]:
print(train.info())
print(train.shape)
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          2126 non-null   float64
 1   accelerations                                           2126 non-null   float64
 2   fetal_movement                                          2126 non-null   float64
 3   uterine_contractions                                    2126 non-null   float64
 4   light_decelerations                                     2126 non-null   float64
 5   severe_decelerations                                    2126 non-null   float64
 6   prolongued_decelerations                                2126 non-null   float64
 7   abnormal_short_term_variability                         2126 non-null   float64
 8   mean_value_of_short_term_variability  

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [61]:
train.describe()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
count,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.00000,...,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000
mean,133.303857,0.003178,0.009481,0.004366,0.001889,0.000003,0.000159,46.990122,1.332785,9.84666,...,93.579492,164.025400,4.068203,0.323612,137.452023,134.610536,138.090310,18.808090,0.320320,1.304327
std,9.840844,0.003866,0.046666,0.002946,0.002960,0.000057,0.000590,17.192814,0.883241,18.39688,...,29.560212,17.944183,2.949386,0.706059,16.381289,15.593596,14.466589,28.977636,0.610829,0.614377
min,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.200000,0.00000,...,50.000000,122.000000,0.000000,0.000000,60.000000,73.000000,77.000000,0.000000,-1.000000,1.000000
25%,126.000000,0.000000,0.000000,0.002000,0.000000,0.000000,0.000000,32.000000,0.700000,0.00000,...,67.000000,152.000000,2.000000,0.000000,129.000000,125.000000,129.000000,2.000000,0.000000,1.000000
50%,133.000000,0.002000,0.000000,0.004000,0.000000,0.000000,0.000000,49.000000,1.200000,0.00000,...,93.000000,162.000000,3.000000,0.000000,139.000000,136.000000,139.000000,7.000000,0.000000,1.000000
75%,140.000000,0.006000,0.003000,0.007000,0.003000,0.000000,0.000000,61.000000,1.700000,11.00000,...,120.000000,174.000000,6.000000,0.000000,148.000000,145.000000,148.000000,24.000000,1.000000,1.000000
max,160.000000,0.019000,0.481000,0.015000,0.015000,0.001000,0.005000,87.000000,7.000000,91.00000,...,159.000000,238.000000,18.000000,10.000000,187.000000,182.000000,186.000000,269.000000,1.000000,3.000000


In [62]:
train.corr()['fetal_health']

baseline value                                            0.148151
accelerations                                            -0.364066
fetal_movement                                            0.088010
uterine_contractions                                     -0.204894
light_decelerations                                       0.058870
severe_decelerations                                      0.131934
prolongued_decelerations                                  0.484859
abnormal_short_term_variability                           0.471191
mean_value_of_short_term_variability                     -0.103382
percentage_of_time_with_abnormal_long_term_variability    0.426146
mean_value_of_long_term_variability                      -0.226797
histogram_width                                          -0.068789
histogram_min                                             0.063175
histogram_max                                            -0.045265
histogram_number_of_peaks                                -0.02

In [63]:
train['fetal_health'].value_counts()

fetal_health
1.0    1655
2.0     295
3.0     176
Name: count, dtype: int64

### Red neuronal 1.0 (Sin tratamiento)

In [64]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

In [65]:
X = train.drop(['fetal_health'], axis=1)
y = train['fetal_health']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [67]:
model = Sequential()

model.add(Dense(64, input_dim=21, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [68]:
y_train = y_train - 1
y_test = y_test - 1
y_val = y_val - 1

In [69]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/50
20/20 [==============================] - 1s 15ms/step - loss: 13.0529 - accuracy: 0.5976 - val_loss: 4.9006 - val_accuracy: 0.5600
Epoch 2/50
20/20 [==============================] - 0s 4ms/step - loss: 3.1958 - accuracy: 0.5804 - val_loss: 1.1595 - val_accuracy: 0.7812
Epoch 3/50
20/20 [==============================] - 0s 3ms/step - loss: 1.0057 - accuracy: 0.7757 - val_loss: 0.8649 - val_accuracy: 0.7906
Epoch 4/50
20/20 [==============================] - 0s 3ms/step - loss: 0.6825 - accuracy: 0.7953 - val_loss: 0.7244 - val_accuracy: 0.6941
Epoch 5/50
20/20 [==============================] - 0s 3ms/step - loss: 0.6058 - accuracy: 0.8047 - val_loss: 0.5680 - val_accuracy: 0.8306
Epoch 6/50
20/20 [==============================] - 0s 3ms/step - loss: 0.4956 - accuracy: 0.8384 - val_loss: 0.5224 - val_accuracy: 0.8329
Epoch 7/50
20/20 [==============================] - 0s 3ms/step - loss: 0.4669 - accuracy: 0.8431 - val_loss: 0.6170 - val_accuracy: 0.8447
Epoch 8/50
20/20 [

In [70]:
loss, accuracy = model.evaluate(X_test, y_test)
print(accuracy)
loss

14/14 [==============================] - 0s 1ms/step - loss: 0.3183 - accuracy: 0.9061
0.9061033129692078


0.3183370530605316

In [71]:
y_pred = model.predict(X_test)
y_pred_clase = y_pred.argmax(axis=1)

14/14 [==============================] - 0s 1ms/step


In [72]:
matrix = confusion_matrix(y_test, y_pred_clase)
print(matrix)

[[322   7   4]
 [ 22  40   2]
 [  1   4  24]]


### Red neuronal 1.1 (variables escaladas)

In [73]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

In [74]:
model2 = Sequential()

model2.add(Dense(64, input_dim=21, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(3, activation='softmax'))

In [75]:
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model2.fit(X_train_scaled, y_train, epochs=50, batch_size=64, validation_data=(X_val_scaled, y_val))

Epoch 1/50
20/20 [==============================] - 1s 15ms/step - loss: 0.8678 - accuracy: 0.7624 - val_loss: 0.6928 - val_accuracy: 0.7953
Epoch 2/50
20/20 [==============================] - 0s 4ms/step - loss: 0.6630 - accuracy: 0.7718 - val_loss: 0.5869 - val_accuracy: 0.7953
Epoch 3/50
20/20 [==============================] - 0s 3ms/step - loss: 0.5956 - accuracy: 0.7718 - val_loss: 0.5413 - val_accuracy: 0.7953
Epoch 4/50
20/20 [==============================] - 0s 3ms/step - loss: 0.5383 - accuracy: 0.7718 - val_loss: 0.5009 - val_accuracy: 0.7953
Epoch 5/50
20/20 [==============================] - 0s 3ms/step - loss: 0.4868 - accuracy: 0.7835 - val_loss: 0.4619 - val_accuracy: 0.8141
Epoch 6/50
20/20 [==============================] - 0s 3ms/step - loss: 0.4390 - accuracy: 0.8071 - val_loss: 0.4271 - val_accuracy: 0.8376
Epoch 7/50
20/20 [==============================] - 0s 3ms/step - loss: 0.3979 - accuracy: 0.8541 - val_loss: 0.3975 - val_accuracy: 0.8541
Epoch 8/50
20/20 [=

In [76]:
loss2, accuracy2 = model2.evaluate(X_test_scaled, y_test)
print(accuracy2)
loss2

14/14 [==============================] - 0s 1ms/step - loss: 0.2651 - accuracy: 0.8967
0.8967136144638062


0.26511579751968384

In [77]:
y_pred2 = model2.predict(X_test_scaled)
y_pred2_clase = y_pred2.argmax(axis=1)

14/14 [==============================] - 0s 1ms/step


In [78]:
matrix2 = confusion_matrix(y_test, y_pred2_clase)
print(matrix2)

[[317  16   0]
 [ 19  43   2]
 [  3   4  22]]


### Red neuronal 1.2 (Con dropout, BatchNormalization y capa densa extra)

In [79]:
model3 = Sequential()

model3.add(Dense(128, input_dim=21, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(Dense(64, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(Dense(32, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(Dense(16, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(Dense(3, activation='softmax'))

In [80]:
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model3.fit(X_train_scaled, y_train, epochs=50, batch_size=64, validation_data=(X_val_scaled, y_val))

Epoch 1/50
20/20 [==============================] - 2s 20ms/step - loss: 1.1942 - accuracy: 0.4510 - val_loss: 1.0782 - val_accuracy: 0.1365
Epoch 2/50
20/20 [==============================] - 0s 7ms/step - loss: 0.9558 - accuracy: 0.5796 - val_loss: 1.0147 - val_accuracy: 0.6353
Epoch 3/50
20/20 [==============================] - 0s 5ms/step - loss: 0.7731 - accuracy: 0.6941 - val_loss: 0.9197 - val_accuracy: 0.7953
Epoch 4/50
20/20 [==============================] - 0s 5ms/step - loss: 0.6836 - accuracy: 0.7396 - val_loss: 0.8166 - val_accuracy: 0.7953
Epoch 5/50
20/20 [==============================] - 0s 5ms/step - loss: 0.6103 - accuracy: 0.8055 - val_loss: 0.7314 - val_accuracy: 0.7953
Epoch 6/50
20/20 [==============================] - 0s 5ms/step - loss: 0.5736 - accuracy: 0.8243 - val_loss: 0.6666 - val_accuracy: 0.7953
Epoch 7/50
20/20 [==============================] - 0s 5ms/step - loss: 0.5018 - accuracy: 0.8392 - val_loss: 0.6127 - val_accuracy: 0.7953
Epoch 8/50
20/20 [=

In [81]:
loss3, accuracy3 = model3.evaluate(X_test_scaled, y_test)
print(accuracy3)
loss3

14/14 [==============================] - 0s 2ms/step - loss: 0.2461 - accuracy: 0.8944
0.8943662047386169


0.2460983693599701

In [82]:
y_pred3 = model3.predict(X_test_scaled)
y_pred3_clase = y_pred3.argmax(axis=1)

14/14 [==============================] - 0s 2ms/step


In [83]:
matrix3 = confusion_matrix(y_test, y_pred3_clase)
print(matrix3)

[[312  20   1]
 [ 14  44   6]
 [  1   3  25]]


### Red neuronal 1.3 (Con función de activación swish)

In [84]:
model4 = Sequential()

model4.add(Dense(128, input_dim=21, activation='swish'))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))
model4.add(Dense(64, activation='swish'))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))
model4.add(Dense(32, activation='swish'))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))
model4.add(Dense(16, activation='swish'))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))
model4.add(Dense(3, activation='softmax'))

In [85]:
model4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model4.fit(X_train_scaled, y_train, epochs=50, batch_size=64, validation_data=(X_val_scaled, y_val))

Epoch 1/50
20/20 [==============================] - 3s 24ms/step - loss: 1.2170 - accuracy: 0.4894 - val_loss: 0.9528 - val_accuracy: 0.7953
Epoch 2/50
20/20 [==============================] - 0s 7ms/step - loss: 0.8756 - accuracy: 0.6376 - val_loss: 0.8127 - val_accuracy: 0.7953
Epoch 3/50
20/20 [==============================] - 0s 6ms/step - loss: 0.7707 - accuracy: 0.7090 - val_loss: 0.6995 - val_accuracy: 0.7953
Epoch 4/50
20/20 [==============================] - 0s 6ms/step - loss: 0.6840 - accuracy: 0.7561 - val_loss: 0.6346 - val_accuracy: 0.7953
Epoch 5/50
20/20 [==============================] - 0s 6ms/step - loss: 0.5943 - accuracy: 0.8000 - val_loss: 0.6092 - val_accuracy: 0.7953
Epoch 6/50
20/20 [==============================] - 0s 6ms/step - loss: 0.5370 - accuracy: 0.8314 - val_loss: 0.5988 - val_accuracy: 0.7953
Epoch 7/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4804 - accuracy: 0.8463 - val_loss: 0.6009 - val_accuracy: 0.7953
Epoch 8/50
20/20 [=

In [86]:
loss4, accuracy4 = model4.evaluate(X_test_scaled, y_test)
print(accuracy4)
loss4

14/14 [==============================] - 0s 2ms/step - loss: 0.2220 - accuracy: 0.9108
0.9107981324195862


0.22198963165283203

In [87]:
y_pred4 = model4.predict(X_test_scaled)
y_pred4_clase = y_pred4.argmax(axis=1)

14/14 [==============================] - 0s 2ms/step


In [88]:
matrix4 = confusion_matrix(y_test, y_pred4_clase)
print(matrix4)

[[323  10   0]
 [ 19  44   1]
 [  4   4  21]]


### Red neuronal 1.4 (Con balanceo de clases)

In [89]:
model5 = Sequential()

model5.add(Dense(128, input_dim=21, activation='swish'))
model5.add(BatchNormalization())
model5.add(Dropout(0.2))
model5.add(Dense(64, activation='swish'))
model5.add(BatchNormalization())
model5.add(Dropout(0.2))
model5.add(Dense(32, activation='swish'))
model5.add(BatchNormalization())
model5.add(Dropout(0.2))
model5.add(Dense(16, activation='swish'))
model5.add(BatchNormalization())
model5.add(Dropout(0.2))
model5.add(Dense(3, activation='softmax'))

In [90]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))
class_weights_dict

{0: 0.43191056910569103, 1: 2.4285714285714284, 2: 3.663793103448276}

In [91]:
model5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model5.fit(X_train_scaled, y_train, epochs=50, batch_size=64, validation_data=(X_val_scaled, y_val), class_weight=class_weights_dict)

Epoch 1/50
20/20 [==============================] - 2s 26ms/step - loss: 1.1699 - accuracy: 0.3914 - val_loss: 1.0245 - val_accuracy: 0.7506
Epoch 2/50
20/20 [==============================] - 0s 7ms/step - loss: 0.8538 - accuracy: 0.5184 - val_loss: 0.9760 - val_accuracy: 0.7906
Epoch 3/50
20/20 [==============================] - 0s 7ms/step - loss: 0.8089 - accuracy: 0.5600 - val_loss: 0.9339 - val_accuracy: 0.8165
Epoch 4/50
20/20 [==============================] - 0s 7ms/step - loss: 0.7247 - accuracy: 0.6047 - val_loss: 0.8783 - val_accuracy: 0.8188
Epoch 5/50
20/20 [==============================] - 0s 7ms/step - loss: 0.6612 - accuracy: 0.6682 - val_loss: 0.8217 - val_accuracy: 0.8118
Epoch 6/50
20/20 [==============================] - 0s 7ms/step - loss: 0.5697 - accuracy: 0.7051 - val_loss: 0.7701 - val_accuracy: 0.8094
Epoch 7/50
20/20 [==============================] - 0s 7ms/step - loss: 0.6136 - accuracy: 0.7380 - val_loss: 0.7227 - val_accuracy: 0.8094
Epoch 8/50
20/20 [=

In [92]:
loss5, accuracy5 = model5.evaluate(X_test_scaled, y_test)
print(accuracy5)
loss5

14/14 [==============================] - 0s 2ms/step - loss: 0.4395 - accuracy: 0.8333
0.8333333134651184


0.43954357504844666

In [93]:
y_pred5 = model5.predict(X_test_scaled)
y_pred5_clase = y_pred5.argmax(axis=1)

14/14 [==============================] - 0s 3ms/step


In [94]:
matrix5 = confusion_matrix(y_test, y_pred5_clase)
print(matrix5)

[[272  55   6]
 [  2  55   7]
 [  1   0  28]]


### Comparación de modelos  
El mejor modelo es el modelo realizado con la funcion swish, auqnue no hay una gran diferencia.  
Si interesa catalogar bien, sobre todo, los casos sospechosos o patologicos, el modelo con balanceo de clases funciona mejor en esas dos categorías, aunque pierde precisión general.

In [99]:
model_data = {
    'Modelo': ['Modelo 1 (Modelo Base)', 'Modelo 2 (Escalado)', 'Modelo 3 (Dropout, Batch y Capa densa extra)', 'Modelo 4 (Función Swish)', 'Modelo 5 (Balanceo de clases)'],
    'Accuracy entrenamiento': [0.8776, 0.9184, 0.9216, 0.9231, 0.8863],
    'Accuracy prueba': [0.9061, 0.8967, 0.8943, 0.9107, 0.8333],
    'Loss entrenamiento': [0.3, 0.19, 0.19, 0.21, 0.31],
    'Loss prueba': [0.32, 0.27, 0.25, 0.22, 0.44]}
df = pd.DataFrame(model_data)

In [101]:
df

,Modelo,Accuracy entrenamiento,Accuracy prueba,Loss entrenamiento,Loss prueba
0,Modelo 1 (Modelo Base),0.8776,0.9061,0.30,0.32
1,Modelo 2 (Escalado),0.9184,0.8967,0.19,0.27
2,"Modelo 3 (Dropout, Batch y Capa densa extra)",0.9216,0.8943,0.19,0.25
3,Modelo 4 (Función Swish),0.9231,0.9107,0.21,0.22
4,Modelo 5 (Balanceo de clases),0.8863,0.8333,0.31,0.44
